In [2]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords
import re
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot

In [3]:
!conda list | grep pandas

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
df=pd.read_csv('video_captioning_dataset/video_corpus.csv')

In [4]:
df.head()

,VideoID,Start,End,WorkerID,Source,AnnotationTime,Language,Description
0,mv89psg6zh4,33,46,588702,unverified,55,Slovene,Papagaj se umiva pod tekočo vodo v lijaku.
1,mv89psg6zh4,33,46,588702,unverified,37,Slovene,Papagaj se umiva pod tekočo vodo v lijaku.
2,mv89psg6zh4,33,46,362812,unverified,11,Macedonian,папагал се бања
3,mv89psg6zh4,33,46,968828,unverified,84,German,Ein Wellensittich duscht unter einem Wasserhahn.
4,mv89psg6zh4,33,46,203142,unverified,14,Romanian,o pasare sta intr-o chiuveta.


In [5]:
df.describe()

,Start,End,WorkerID,AnnotationTime
count,122604.000000,122604.000000,122604.000000,122604.000000
mean,77.370143,87.319337,526368.946649,79.590560
std,118.899799,118.361142,261638.665266,176.854277
min,0.000000,3.000000,100234.000000,5.000000
25%,7.000000,18.000000,297776.000000,24.000000
50%,32.000000,42.000000,553878.000000,39.000000
75%,98.000000,108.000000,760882.000000,69.000000
max,997.000000,1004.000000,998645.000000,3590.000000


In [6]:
df.shape

(122604, 8)

In [7]:
df['VideoID'].nunique()
df['Language'].nunique()

35

In [8]:
import os,os.path

In [9]:
path,dirs,files=next(os.walk("video_captioning_dataset/YouTubeClips/YouTubeClips"))
file_count=len(files)

In [10]:
print(file_count)

1969


In [11]:
# process video captions in english language
is_eng =  df['Language']=='English'
df_eng = df[is_eng]

In [12]:
df_eng.shape

(85511, 8)

In [13]:
df_eng.head()

,VideoID,Start,End,WorkerID,Source,AnnotationTime,Language,Description
18,mv89psg6zh4,33,46,682611,clean,66,English,A bird in a sink keeps getting under the runni...
19,mv89psg6zh4,33,46,760882,clean,16,English,A bird is bathing in a sink.
20,mv89psg6zh4,33,46,878566,clean,76,English,A bird is splashing around under a running fau...
21,mv89psg6zh4,33,46,707318,clean,14,English,A bird is bathing in a sink.
22,mv89psg6zh4,33,46,135621,clean,58,English,A bird is standing in a sink drinking water th...


In [14]:
len(df_eng)

85511

In [15]:
print(df_eng['VideoID'].iloc[0])

mv89psg6zh4


In [16]:
descriptions={}
for i in range(len(df_eng)):
    if df_eng['VideoID'].iloc[i] not in descriptions:
        descriptions[df_eng['VideoID'].iloc[i]]=[]
    descriptions[df_eng['VideoID'].iloc[i]].append(df_eng['Description'].iloc[i])
    #if i==5:
       # break
       

In [17]:
print(len(descriptions))

1586


In [42]:
#descriptions['-4wsuPCjDBc']
#['mv89psg6zh4']

["A squirrel is eating a peanut in it's shell.",
 'A chipmunk is eating.',
 'A chipmunk is eating a peanut.',
 'A chipmunk is eating a nut.',
 'A squirrel is eating a nut.',
 'A squirrel is eating a whole peanut.',
 'A squirrel is eating a peanut.',
 'A squirrel=l is eating nuts.',
 'A small squirrel is eating a peanut.',
 'A small animal is chewing on a nut.',
 'An ardilla is eating.',
 'A squirrel is eating a nut.',
 'A chipmunk is eating some food.',
 'The squirrel is eating.',
 'A ferret eats a leaf.',
 'A hamster is eating a peanut.',
 'A ardilla coreana eating.',
 'the rabbit is eating',
 'A chipmunk is eating a peanut.',
 'A rabbit is eating',
 'a rabbit is eating',
 'a squirrel eating a fruit',
 'A chipmunk is eating.',
 'A man is playing guitar',
 'A squirrel or some such animal eating a nut of some kind.',
 'A chipmunk is eating some food.',
 'A rat is playing and eating something.',
 'The squirrel ate the peanut out of the shell.',
 'The squirrel is eating.']

In [19]:
# view the first two items in teh dictionary
descriptions_items=descriptions.items()
#list(descriptions_items)[:2]
# get the clean list of video_id's
my_videos=list(descriptions.keys())
#print(my_videos[1:5])

#copy all the files with english captions to a diff folder for each of the training,testing and validation sets
source_folder='video_captioning_dataset/YouTubeClips/YouTubeClips'
dest_folder = 'video_captioning_dataset/YouTubeClips/Eng_YouTubeClips'
for folders, subfolders, filenames in os.walk(source_folder):
    for filename in filenames:
        nfilename=filename.split('_')[0]
        if nfilename in my_videos:
            shutil.copy(os.path.join(folders, filename), dest_folder)
    


In [20]:
# check the count of files in the new folder
path,dirs,files=next(os.walk("video_captioning_dataset/YouTubeClips/Eng_YouTubeClips"))
file_count=len(files)
print(file_count)

1613


In [21]:
# split data set into train and test sets
allfilenames=os.listdir("video_captioning_dataset/YouTubeClips/Eng_YouTubeClips")
np.random.shuffle(allfilenames)
#allfilenames[1:5]
train_filenames, val_filenames, test_filenames = np.split(np.array(allfilenames),
                                                          [int(len(allfilenames)*0.8), int(len(allfilenames)*0.95)])


In [22]:
#print('Total images:',len(allfilenames))
#print('Training:',len(train_filenames))
#print('validation:',len(val_filenames))
#print('Test:',len(test_filenames))
#copy files into diff folders
for name in train_filenames:
    shutil.copy(os.path.join(dest_folder,name),"video_captioning_dataset/YouTubeClips/Training_clips")
    
for name in val_filenames:
    shutil.copy(os.path.join(dest_folder,name),"video_captioning_dataset/YouTubeClips/Validation_clips")
for name in test_filenames:
    shutil.copy(os.path.join(dest_folder,name),"video_captioning_dataset/YouTubeClips/Testing_clips")    

In [64]:
# seperate captions for train, validate and test data sets
#training_captions={x:data[x] for x in descriptions if x in trainingfilenames}
import csv
sourcefolder=os.listdir("video_captioning_dataset/YouTubeClips/Training_clips")
training_captions={}

for filename in sourcefolder:
    nfilename=filename.split('_')[0]
    #print(nfilename)
    training_captions[nfilename]=descriptions[nfilename]

 

# with open('training_text.csv', 'w') as f:
#    #training_text.truncate()
#    for key in training_captions.keys():
#        f.write("%s,%s\n"%(key,training_captions[key]))
#f.close()        
        

In [112]:
from PIL import Image
from keras.preprocessing import image
import cv2

os.getcwd()
def load_image(path):
    """ Load image from path. Return a numpy array """
    image = Image.open(path)
    return np.asarray(image) / 255

def show_video(idx,train):
    if train:
        imagefolder='video_captioning_dataset/YouTubeClips/Training_clips'
        images_files=os.listdir("video_captioning_dataset/YouTubeClips/Training_clips")
        filename=images_files[idx]
        imageid=filename.split('_')[0]
        captions=training_captions[imageid]
        for caption in captions:
            print("caption")
        path=os.path.join(imagefolder,filename)        
        cap=cv2.VideoCapture(path)
        
        #print(filename)
        while(cap.isOpened()):
            ret,frame=cap.read()
            if ret == True:
                cv2.imshow('Frame',frame) 
                if cv2.waitKey(25) & 0xFF==ord('q'):
                    break
        #else:
           # print("error in  loading the video file")
           # break
    #else:
       # break
        

In [ ]:
show_video(idx=2,train=True)

caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption
caption


In [93]:
#cleaning
#from string import digits

def clean_text(string):
    string=str(string)
    string=re.sub("[^a-zA-Z]"," ",string)
    string=re.sub("[^a-zA-Z]"," ",string)
    string=string.lower()
    string=string.split()
    string=[s for s in string if len(s)>1]
    string=[s for s in string if s not in stopwords.words('english')]
    string=" ".join(string)
    return string

In [44]:
clean_text("A cockatiel in a kitchen$5% sink 789playing under the running water 6.5.")

'cockatiel kitchen sink playing running water'

In [45]:
for x in descriptions:
    print(descriptions[x])
    break

['A bird in a sink keeps getting under the running water from a faucet.', 'A bird is bathing in a sink.', 'A bird is splashing around under a running faucet.', 'A bird is bathing in a sink.', 'A bird is standing in a sink drinking water that is pouring out of the facet.', 'A faucet is running while a bird stands in the sink below.', 'A bird is playing in a sink with running water.', 'A bird is playing in tap water.', 'A bird is bathing in the sink.', 'A bird is taking a bath.', 'A bird is taking a shower in a sink.', 'A bird is showering in the sink.', 'The bird is taking a bath under the faucet.', 'A parakeet is taking a shower in a sink.', 'A bird gets washed.', 'A bird is having a bath in the kitchen sink.', 'A bird is bathing in a sink under running water.', 'the bird is drinking water', 'A bird walks under a running tap in a sink.', 'a parrot is playing inside the wash basin.', 'a bird in a sink n playing with water', 'A bird is playing in a sink.', 'A cockatiel in a kitchen sink 

In [62]:
# modify the descriptions of all teh video text
for key,desc_list in descriptions.items():
    for i in range(len(desc_list)):
        desc_list[i]=clean_text(desc_list[i])
            #print(desc_list[i])
   
            
   

In [78]:
len(vocabulary)

13681

In [25]:
list_descriptions=df_eng.Description.tolist()

In [26]:
len(list_descriptions)

85511

In [27]:
clean_list= list()
for i in range(len(list_descriptions)):
    clean_string=clean_text(list_descriptions[i])
    clean_list.append(clean_string)

In [28]:
clean_list[2500]

'lady peeled eggplant'

In [35]:
# find unique vocabulaty
temp=clean_list[2500]
words=clean_list[2500].split(' ')
len(clean_list)
len(words)

3

In [36]:
# total distinct words
vocabulary=set()
for i in range(len(clean_list)):
    vocabulary.update(clean_list[i].split(' '))
#vocabulary size
len(vocabulary)


12756

In [38]:
voc_size=12756
onehot_repr=[one_hot(words,voc_size)for words in clean_list]

In [41]:
clean_list[0:5]

['bird sink keeps getting running water faucet',
 'bird bathing sink',
 'bird splashing around running faucet',
 'bird bathing sink',
 'bird standing sink drinking water pouring facet']

In [42]:
# print one hot representation of words
onehot_repr[0:5]

[[11528, 8830, 11081, 995, 10263, 11466, 1358],
 [11528, 10898, 8830],
 [11528, 771, 7261, 10263, 1358],
 [11528, 10898, 8830],
 [11528, 6208, 8830, 4873, 11466, 7232, 10324]]

In [45]:
#Word Embedding Representation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np

In [51]:
max_len=-1
for ele in clean_list:
    if len(ele)>max_len:
        max_len=len(ele)
        res=ele
len(res.split(' '))  

88

In [52]:
#padding one hot representation using pre method
sent_length=88
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [53]:
embedded_docs[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
       11528,  8830, 11081,   995, 10263, 11466,  1358])

In [ ]:
# taking a dimension of 100 
dim=100

In [54]:
model=Sequential()
model.add(Embedding(voc_size,100,input_length=sent_length))
model.compile('adam','mse')

In [55]:
print(model.predict(embedded_docs)[0])

array([[-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
        -0.0018641 , -0.01832164],
       [-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
        -0.0018641 , -0.01832164],
       [-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
        -0.0018641 , -0.01832164],
       ...,
       [ 0.03030716,  0.01606182, -0.0065761 , ...,  0.03062374,
        -0.02150539, -0.04024249],
       [ 0.0236884 ,  0.01069669, -0.04002314, ..., -0.02003026,
         0.01693315, -0.00209162],
       [-0.04187291,  0.0068629 ,  0.04735855, ..., -0.03175991,
         0.01724629,  0.01773698]], dtype=float32)

In [56]:
model.predict(embedded_docs)

array([[[-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
         -0.0018641 , -0.01832164],
        [-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
         -0.0018641 , -0.01832164],
        [-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
         -0.0018641 , -0.01832164],
        ...,
        [ 0.03030716,  0.01606182, -0.0065761 , ...,  0.03062374,
         -0.02150539, -0.04024249],
        [ 0.0236884 ,  0.01069669, -0.04002314, ..., -0.02003026,
          0.01693315, -0.00209162],
        [-0.04187291,  0.0068629 ,  0.04735855, ..., -0.03175991,
          0.01724629,  0.01773698]],

       [[-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
         -0.0018641 , -0.01832164],
        [-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
         -0.0018641 , -0.01832164],
        [-0.04588137, -0.00036844,  0.04107795, ...,  0.01081521,
         -0.0018641 , -0.01832164],
        ...,
        [ 0.01540625,  0.00233319, -0.01622102, ..., -